# Building and Answer with index page

In [1]:
import sys
sys.path.append("../")
from src.service.provider import ProviderService

provider = ProviderService()

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


From (44, 5) to 44
From (11, 4) to 11


In [6]:
from langchain_core.prompts import ChatPromptTemplate
from operator import itemgetter


TEMPLATE = """You are a helpful admission assistant for Ton Duc Thang university who gives helpful and accurate information.
Answer the question mainly using the following context. Output "None" if you cannot answer:
```
{context}
```
Question: {question}
"""

TEMPLATE = """
Bạn là một người tư vấn viên thân thiện và đầy hiểu biết. Nhiệm vụ của bạn là hỗ trợ người dùng hiểu biết hơn về trường đại học Tôn Đức Thắng.

Đây là thông tin mà bạn biết:
```
Trường có các ngành đào tạo sau

- Ngành kỹ thuật phần mềm
- Ngành khoa học máy tính
- Ngành Dược
- Ngành Kỹ thuật công trình

```
Hãy hỗ trợ thật tốt với yêu cầu sau đến từ người dùng.
Yêu cầu: {question}
"""


gemini = provider.get_gemini_pro(False)
prompt = ChatPromptTemplate.from_template(TEMPLATE)

chain = ({
    "question": itemgetter("question")
    } 
    | prompt 
    | gemini)

In [4]:
q = "cho em hỏi trường có bao nhiêu ngành ạ"
a = chain.invoke({"question": q})
a

AIMessage(content='Chào bạn,\n\nTôi hiểu bạn đang muốn biết Trường Đại học Tôn Đức Thắng có bao nhiêu ngành đào tạo. Theo thông tin tôi có, trường hiện đang đào tạo **4 ngành** chính, bao gồm:\n\n1. Ngành Kỹ thuật phần mềm\n2. Ngành Khoa học máy tính\n3. Ngành Dược\n4. Ngành Kỹ thuật công trình\n\nNếu bạn có bất kỳ câu hỏi nào khác về Trường Đại học Tôn Đức Thắng, xin vui lòng cho tôi biết. Tôi luôn sẵn lòng hỗ trợ bạn.')

In [7]:
q = "cho em hỏi trường có bao nhiêu ngành ạ"
q = "trường có đạo tạo ngành IT không ạ"
a = chain.invoke({"question": q})
a


AIMessage(content='Chào bạn,\n\nCảm ơn bạn đã quan tâm đến Trường Đại học Tôn Đức Thắng.\n\nĐúng vậy, trường có đào tạo ngành liên quan đến IT, cụ thể là:\n\n* **Ngành Khoa học máy tính**\n* **Ngành Kỹ thuật phần mềm**\n\nHai ngành này đều cung cấp kiến thức và kỹ năng toàn diện về lĩnh vực công nghệ thông tin, bao gồm:\n\n* Thiết kế và phát triển phần mềm\n* Phân tích dữ liệu\n* Trí tuệ nhân tạo\n* An ninh mạng\n\nNếu bạn quan tâm đến việc theo đuổi sự nghiệp trong lĩnh vực IT, Trường Đại học Tôn Đức Thắng là một lựa chọn tuyệt vời. Trường có đội ngũ giảng viên giàu kinh nghiệm, cơ sở vật chất hiện đại và các chương trình hợp tác với các công ty công nghệ hàng đầu.\n\nHãy cho tôi biết nếu bạn có bất kỳ câu hỏi nào khác. Tôi rất vui được hỗ trợ bạn.')

# Test

In [2]:
from src.prepare.data_load import DocDataLoader
from src.chain.followup import LocalFollowUpChain
doc_loader = DocDataLoader()
config = Configuration()
config.enable_tracing(project="LEARN")

docs = doc_loader.load_major_docs_full()
d1 = docs[15]
d2 = docs[6]
print(d1.page_content[:30])
print(d2.page_content[:30])

From (44, 4) to 44
Kỹ thuật hoá học là ngành học
Công nghệ sinh học là ngành ư


In [3]:
config.enable_tracing(project="LEARN")
followup = LocalFollowUpChain(config=Configuration(), doc_content=d1.page_content)

In [4]:
q = "cho em hỏi chuyên ngành này tên là gì?"
a = followup.answer(question=q)
a

 followup.log - ERROR - InvokeChainError
Traceback (most recent call last):
  File "/home/h4438/Desktop/graduate/WebQA/core/experiments/../src/chain/followup.py", line 74, in answer
    response = self.chain.invoke(inputs)
  File "/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/langchain_core/runnables/base.py", line 2053, in invoke
    input = step.invoke(
  File "/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/langchain_core/runnables/base.py", line 2692, in invoke
    output = {key: future.result() for key, future in zip(steps, futures)}
  File "/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/langchain_core/runnables/base.py", line 2692, in <dictcomp>
    output = {key: future.result() for key, future in zip(steps, futures)}
  File "/home/h4438/miniconda3/envs/uni/lib/python3.9/concurrent/futures/_base.py", line 446, in result
    return self.__get_result()
  File "/home/h4438/miniconda3/envs/uni/lib/python3.9/concurrent/futures/_base.py", line 39

'[Followup] Sorry! something went wrong'

In [6]:
followup.memory.load_memory_variables({})

{'history': [HumanMessage(content='Ngành học này tên là gì?'),
  AIMessage(content='Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch')]}

In [7]:
q = "Vậy cử nhân ngành này có thể ứng tuyển vào vị trí nào?"
a = followup.answer(question=q)
a

AIMessage(content='Cử nhân Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch có thể ứng tuyển vào các vị trí như:\n- Chuyên viên cung cấp dịch vụ du lịch tại các khu du lịch, điểm vui chơi tham quan du lịch\n- Chuyên gia tư vấn về chiến lược kinh doanh, khai thác, xây dựng và cung cấp sản phẩm du lịch\n- Chuyên viên thuộc Bộ và Sở văn hóa – Thể thao – Du lịch như Trung tâm bồi dưỡng nghiệp vụ du lịch, Viện nghiên cứu và phát triển du lịch, Quy hoạch du lịch, Trung tâm xúc tiến du lịch\n- Tự tạo lập doanh nghiệp du lịch mới với quy mô nhỏ hoặc tham gia giảng dạy tại các cơ sở đào tạo về du lịch')

In [23]:
a.content

'Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch'

In [8]:
followup.memory.load_memory_variables({})

{'history': [HumanMessage(content='Ngành học này tên là gì?'),
  AIMessage(content='Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch'),
  HumanMessage(content='Vậy cử nhân ngành này có thể ứng tuyển vào vị trí nào?'),
  AIMessage(content='Cử nhân Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch có thể ứng tuyển vào các vị trí như:\n- Chuyên viên cung cấp dịch vụ du lịch tại các khu du lịch, điểm vui chơi tham quan du lịch\n- Chuyên gia tư vấn về chiến lược kinh doanh, khai thác, xây dựng và cung cấp sản phẩm du lịch\n- Chuyên viên thuộc Bộ và Sở văn hóa – Thể thao – Du lịch như Trung tâm bồi dưỡng nghiệp vụ du lịch, Viện nghiên cứu và phát triển du lịch, Quy hoạch du lịch, Trung tâm xúc tiến du lịch\n- Tự tạo lập doanh nghiệp du lịch mới với quy mô nhỏ hoặc tham gia giảng dạy tại các cơ sở đào tạo về du lịch')]}